In [1]:
import os
import requests
import pandas as pd
import json
import datetime
from dotenv import load_dotenv
from pathlib import Path

In [116]:
weather_data = pd.read_csv(
    Path('cleaned_data/df_weather_data.csv')
)

locations = pd.read_csv(
    Path('blmn_locs.csv')
)

load_dotenv()

# Connecting API key in .env to weather_api_key function
weather_api_key = os.getenv("WEATHER_API_KEY")
display(type(weather_api_key))

#Setting Request URL and creating list for responses from API
request_url = "https://api.weatherstack.com/historical"

str

In [46]:
check_df = weather_data[['request.query', 'date']]
check_df['request.query']= check_df['request.query'].apply(str)
check_df['key'] = check_df['request.query'] + check_df['date']
check_list = check_df['key'].to_list()


C:\Users\carra\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\carra\anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [48]:
dates = list()
start_date = datetime.date(2021, 1, 1)
numdays = 59
for x in range (0, numdays):
    dates.append((start_date + datetime.timedelta(days = x)).strftime("%Y-%m-%d"))

In [49]:
zip_codes = locations['zip_code'].unique()
zip_codes = [str(int) for int in zip_codes]
len(zip_codes)

817

In [50]:
check_key = []

for zip in zip_codes:
    for date in dates:
        x = zip + date
        check_key.append(x)

    

In [51]:
len(check_key)

48203

In [52]:
def non_match_elements(check_key, check_list):
    non_match = []
    for i in check_key:
        if i not in check_list:
            non_match.append(i)
    return non_match

In [63]:
non_match = non_match_elements(check_key, check_list)
len(non_match)

4323

In [64]:
unique_key = check_df['key'].unique()
len(unique_key)

43880

In [102]:
to_add_date = []
to_add_zip = []
len(non_match[0])

15

In [104]:
for i in non_match:
    x = i[-10:]
    to_add_date.append(x)

In [105]:
for i in non_match:
    x = len(i) - 10
    y = i[:x]
    to_add_zip.append(y)
    


In [119]:
responses = []


In [121]:

for i in non_match:
    x = len(i) - 10
    date_api = i[-10:]
    zip_api = i[:x]
    
    params = dict(
    access_key=weather_api_key,
    query= zip_api,
    historical_date = date_api,
    hourly=1, 
    interval=24)
    
    
    rr= requests.get(request_url, params=params)
    data = json.loads(rr.text)
    responses.append(data)

In [124]:
with open('api_responses/weather_data_missing_1102022.json', 'w') as json_file:
    json.dump(responses, json_file)

In [135]:
df_weather_data = pd.read_csv(
    Path('cleaned_data/df_weather_data.csv')
)
df_weather_data_add = pd.read_csv(
    Path('cleaned_data/df_weather_data_01102022.csv')
)
    

In [136]:
df_weather_data.tail()

,Unnamed: 0,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,location.lon,...,dewpoint,windchill,windgust,feelslike,date,mintemp,maxtemp,avgtemp,totalsnow,sunhour
47653,47653,Zipcode,32812,en,m,Orlando,USA,Florida,28.490,-81.320,...,19,24,21,25,2021-02-28,20,29,25,0.0,11.6
47654,47654,Zipcode,32821,en,m,Orlando,USA,Florida,28.379,-81.468,...,18,24,22,26,2021-02-28,20,29,25,0.0,11.6
47655,47655,Zipcode,33418,en,m,Palm Beach Gardens,USA,Florida,26.860,-80.140,...,20,25,33,27,2021-02-28,24,26,25,0.0,8.7
47656,47656,Zipcode,32504,en,m,Pensacola,USA,Florida,30.490,-87.180,...,19,21,22,22,2021-02-28,20,24,22,0.0,4.2
47657,47657,Zipcode,33322,en,m,Fort Lauderdale,USA,Florida,26.150,-80.270,...,19,26,31,27,2021-02-28,24,28,26,0.0,8.7


In [137]:
df_weather_data = pd.concat([df_weather_data, df_weather_data_add], ignore_index=True, axis=0)

In [138]:
df_weather_data.tail()

,Unnamed: 0,request.type,request.query,request.language,request.unit,location.name,location.country,location.region,location.lat,location.lon,...,dewpoint,windchill,windgust,feelslike,date,mintemp,maxtemp,avgtemp,totalsnow,sunhour
49203,1545,Zipcode,33322,en,m,Fort Lauderdale,USA,Florida,26.15,-80.274,...,19,26,16,27,2021-01-27,22,29,26,0.0,10.9
49204,1546,Zipcode,33322,en,m,Fort Lauderdale,USA,Florida,26.15,-80.274,...,12,21,29,21,2021-01-28,15,22,21,0.0,10.1
49205,1547,Zipcode,33322,en,m,Fort Lauderdale,USA,Florida,26.15,-80.274,...,10,18,27,18,2021-01-29,17,19,18,0.0,7.0
49206,1548,Zipcode,33322,en,m,Fort Lauderdale,USA,Florida,26.15,-80.274,...,11,19,24,19,2021-01-30,18,20,19,0.0,5.5
49207,1549,Zipcode,33322,en,m,Fort Lauderdale,USA,Florida,26.15,-80.274,...,14,22,24,23,2021-01-31,17,24,22,0.0,10.1


In [140]:
#exporting cleaned data to csv
df_weather_data.to_csv('cleaned_data/df_weather_data_updated.csv')